In [1]:
# config.ipynb cell
import os
import sys
import supabase

# Dynamically add the parent directory to the Python path
current_dir = os.path.dirname(os.path.abspath(__name__))
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

import uuid
import random
import json
from faker import Faker
from datetime import datetime
from supabase import create_client, Client
from src.api.config import settings

# Supabase credentials
SUPABASE_URL = settings.SUPABASE_URL
SUPABASE_KEY = settings.SUPABASE_ANON_KEY
client = create_client(SUPABASE_URL, SUPABASE_KEY)


🔍 2. Pick a Random Test User

In [2]:
import random

# Fetch a few users to test
users = client.table("user_profiles").select("user_id").limit(10).execute().data
test_user_id = random.choice(users)["user_id"]
print("🔍 Testing for user_id:", test_user_id)


🔍 Testing for user_id: 876f58a6-7661-42dc-974b-9b0e951181a0


In [14]:
user = client.table("properties").select("*").eq("id", 1).execute().data
user[0]

{'id': 1,
 'owner_user_id': '2f850ed7-8138-4c7f-aaf4-cd4ba2b5c930',
 'address': '4106 Peterson Center, East Matthew, MA 92472',
 'location': 'Guadalajara',
 'price': 4928,
 'amenities': ['parking', 'wifi', 'balcony'],
 'num_rooms': 3,
 'bathrooms': 2,
 'available_from': '2025-07-13T23:28:52.28672',
 'available_to': '2025-12-27T23:28:52.28672',
 'created_at': '2025-07-01T23:28:52.286729+00:00',
 'updated_at': '2025-07-01T23:28:52.286729+00:00'}

🧠 3. Matchmaking Logic

In [64]:
from datetime import datetime

def match_top(user_id: str, top_k: int = 5):
    user = client.table("user_profiles").select("*").eq("user_id", user_id).single().execute().data
    if not user:
        return {"error": "User not found"}

    budget_min = user["budget_min"]
    budget_max = user["budget_max"]
    location = user["location_preference"]
    lifestyle_tags = set(user.get("lifestyle_tags") or [])

    roommates = client.table("user_profiles") \
        .select("*") \
        .neq("user_id", user_id) \
        .eq("location_preference", location) \
        .gte("budget_max", budget_min) \
        .lte("budget_min", budget_max) \
        .execute().data

    properties = client.table("properties") \
        .select("*") \
        .eq("location", location) \
        .gte("price", budget_min) \
        .lte("price", budget_max) \
        .lte("available_from", datetime.utcnow().isoformat()) \
        .execute().data

    def roommate_score(rm):
        rm_tags = set(rm.get("lifestyle_tags") or [])
        tag_score = len(lifestyle_tags & rm_tags) / len(lifestyle_tags | rm_tags) if lifestyle_tags and rm_tags else 0
        rm_budget_avg = (rm["budget_min"] + rm["budget_max"]) / 2
        user_budget_avg = (budget_min + budget_max) / 2
        budget_score = 1 - abs(user_budget_avg - rm_budget_avg) / max(user_budget_avg, rm_budget_avg)
        return round(0.5 * budget_score + 0.5 * tag_score, 3)

    def property_score(prop):
        prop_amenities = set(prop.get("amenities") or [])
        amenity_score = len(lifestyle_tags & prop_amenities) / len(lifestyle_tags | prop_amenities) if lifestyle_tags and prop_amenities else 0
        price_score = 1 - abs(((budget_min + budget_max) / 2) - prop["price"]) / budget_max
        return round(0.7 * price_score + 0.3 * amenity_score, 3)

    top_roommates = sorted(roommates, key=roommate_score, reverse=True)[:top_k]
    top_properties = sorted(properties, key=property_score, reverse=True)[:top_k]

    return {
        "roommate_matches": [
            {"user_id": rm["user_id"], "score": roommate_score(rm)} for rm in top_roommates
        ],
        "property_matches": [
            {"property_id": prop["id"], "score": property_score(prop)} for prop in top_properties
        ]
    }
# Example usage

In [65]:
k = 5
results = match_top(user_id=test_user_id, top_k=k)


In [66]:
results

{'roommate_matches': [{'user_id': 'f5c4f074-a14e-4634-8afb-f2ccdddb3700',
   'score': 0.841},
  {'user_id': '365f5926-b8cb-4037-9b19-de6f913a7545', 'score': 0.818},
  {'user_id': 'ddcfb9bb-24f6-4e05-aaa8-c9b8412ea3b7', 'score': 0.797},
  {'user_id': 'e279cad8-4c42-4486-b189-05d2d60bef1c', 'score': 0.784},
  {'user_id': '9e01d97e-acb0-4e2c-98d4-17b690aed010', 'score': 0.779}],
 'property_matches': [{'property_id': 518, 'score': 0.695},
  {'property_id': 302, 'score': 0.66},
  {'property_id': 738, 'score': 0.626},
  {'property_id': 436, 'score': 0.584}]}

In [67]:
print(f"🔍 Top K roommates and properties for UserID: {test_user_id}\n")
# print for k - 5
print(f"For K = {k}, found {len(results['roommate_matches'])} roommates and {len(results['property_matches'])} properties.\n")

print("🏠 Top Property Matches:")
for idx, match in enumerate(results["property_matches"], start=1):
    print(f"{idx}. Property ID: {match['property_id']}, Match Score: {match['score']}")

print("\n👥 Top Roommate Matches:")
for idx, match in enumerate(results["roommate_matches"], start=1):
    print(f"{idx}. User ID: {match['user_id']}, Match Score: {match['score']}")

🔍 Top K roommates and properties for UserID: b5634cac-b8da-4dd0-8256-51a06003b794

For K = 5, found 5 roommates and 4 properties.

🏠 Top Property Matches:
1. Property ID: 518, Match Score: 0.695
2. Property ID: 302, Match Score: 0.66
3. Property ID: 738, Match Score: 0.626
4. Property ID: 436, Match Score: 0.584

👥 Top Roommate Matches:
1. User ID: f5c4f074-a14e-4634-8afb-f2ccdddb3700, Match Score: 0.841
2. User ID: 365f5926-b8cb-4037-9b19-de6f913a7545, Match Score: 0.818
3. User ID: ddcfb9bb-24f6-4e05-aaa8-c9b8412ea3b7, Match Score: 0.797
4. User ID: e279cad8-4c42-4486-b189-05d2d60bef1c, Match Score: 0.784
5. User ID: 9e01d97e-acb0-4e2c-98d4-17b690aed010, Match Score: 0.779


In [68]:
test_user_id

'b5634cac-b8da-4dd0-8256-51a06003b794'

In [83]:
import requests

url = "https://782e-2806-2f0-4060-f367-90b7-78df-5d0b-de33.ngrok-free.app/matchmaking/match/top"
params = {
    "user_id": test_user_id,
    "top_k": k
}

response_ngrok = requests.post(f"https://782e-2806-2f0-4060-f367-90b7-78df-5d0b-de33.ngrok-free.app/matchmaking/match/top", params=params)

In [89]:
response_ngrok.status_code

200

In [90]:
response_ngrok.headers

{'Content-Length': '503', 'Content-Type': 'application/json', 'Date': 'Wed, 02 Jul 2025 00:52:34 GMT', 'Ngrok-Agent-Ips': '2806:2f0:4060:f367:90b7:78df:5d0b:de33', 'Server': 'uvicorn'}

In [84]:
response_ngrok.json()

{'roommate_matches': [{'user_id': 'f5c4f074-a14e-4634-8afb-f2ccdddb3700',
   'score': 0.841},
  {'user_id': '365f5926-b8cb-4037-9b19-de6f913a7545', 'score': 0.818},
  {'user_id': 'ddcfb9bb-24f6-4e05-aaa8-c9b8412ea3b7', 'score': 0.797},
  {'user_id': 'e279cad8-4c42-4486-b189-05d2d60bef1c', 'score': 0.784},
  {'user_id': '9e01d97e-acb0-4e2c-98d4-17b690aed010', 'score': 0.779}],
 'property_matches': [{'property_id': 518, 'score': 0.695},
  {'property_id': 302, 'score': 0.66},
  {'property_id': 738, 'score': 0.626},
  {'property_id': 436, 'score': 0.584}]}